In [13]:
import numpy as np
import tensorflow as tf
import pandas as pd
import glob
from tqdm import tqdm
from music21 import *
from tensorflow.python.ops import control_flow_ops
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [3]:
s = converter.parse('elise.mid')

In [7]:
notes = []
for file in glob.glob("*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [11]:
n_vocab=len(set(notes))

In [14]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [18]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [19]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=10, batch_size=64, callbacks=callbacks_list)

Epoch 1/10
829/829 [==============================] - ETA: 1:54 - loss: 4.381 - ETA: 1:25 - loss: 4.246 - ETA: 1:11 - loss: 4.298 - ETA: 1:01 - loss: 4.302 - ETA: 54s - loss: 4.276 - ETA: 46s - loss: 4.23 - ETA: 39s - loss: 4.21 - ETA: 32s - loss: 4.18 - ETA: 26s - loss: 4.15 - ETA: 19s - loss: 4.10 - ETA: 12s - loss: 4.07 - ETA: 6s - loss: 4.0380 - 85s 102ms/step - loss: 4.0267
Epoch 2/10
829/829 [==============================] - ETA: 1:17 - loss: 3.584 - ETA: 1:10 - loss: 3.488 - ETA: 1:03 - loss: 3.591 - ETA: 57s - loss: 3.593 - ETA: 50s - loss: 3.58 - ETA: 44s - loss: 3.61 - ETA: 38s - loss: 3.59 - ETA: 31s - loss: 3.58 - ETA: 25s - loss: 3.58 - ETA: 18s - loss: 3.57 - ETA: 12s - loss: 3.55 - ETA: 6s - loss: 3.5835 - 83s 100ms/step - loss: 3.5857
Epoch 3/10
829/829 [==============================] - ETA: 1:17 - loss: 3.572 - ETA: 1:10 - loss: 3.596 - ETA: 1:04 - loss: 3.587 - ETA: 57s - loss: 3.616 - ETA: 51s - loss: 3.56 - ETA: 45s - loss: 3.55 - ETA: 38s - loss: 3.53 - ETA: 32s 

In [29]:
model = Sequential()
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [48]:
# Load the weights to each node
model.load_weights('weights-improvement-09-3.2697-bigger.hdf5')

ValueError: Cannot feed value of shape (1, 1024) for Tensor 'Placeholder_28:0', which has shape '(1, 2048)'

In [45]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
pattern=list(pattern)
# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [46]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [47]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')

'test_output.mid'